In [4]:
import requests
import json
from datetime import datetime
import pandas as pd
import warnings
import os

warnings.filterwarnings('ignore')

In [5]:
#url link from - https://www.redfin.com/news/data-center/
#reads in national zip code level real estate market data
#Duration period: 90

url = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/zip_code_market_tracker.tsv000.gz'


#assign to dataframe
df = pd.read_csv(url, compression='gzip', sep='\t', on_bad_lines='skip')

print('Num of rows:', len(df))
print('Num of cols:', len(df.columns)) 

#preview data
df.head()

Num of rows: 6680997
Num of cols: 58


,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2018-11-01,2019-01-31,90,zip code,2,18999,f,Zip Code: 45226,NaN,Ohio,...,0.000000,0.500000,0.071429,0.166667,0.000000,0.000000,0.000000,"Cincinnati, OH",17140,2022-11-20 14:36:43
1,2020-12-01,2021-02-28,90,zip code,2,3606,f,Zip Code: 11003,NaN,New York,...,0.183093,0.339286,0.028941,-0.026099,0.428571,0.193277,0.345238,"Nassau County, NY",35004,2022-11-20 14:36:43
2,2015-07-01,2015-09-30,90,zip code,2,2151,f,Zip Code: 06107,NaN,Connecticut,...,0.107143,NaN,NaN,NaN,0.500000,NaN,0.500000,"Hartford, CT",25540,2022-11-20 14:36:43
3,2017-03-01,2017-05-31,90,zip code,2,38575,f,Zip Code: 93063,NaN,California,...,-0.008333,0.529412,0.029412,0.003096,0.444444,0.019444,-0.007937,"Oxnard, CA",37100,2022-11-20 14:36:43
4,2013-03-01,2013-05-31,90,zip code,2,39921,f,Zip Code: 95938,NaN,California,...,-0.282828,NaN,NaN,NaN,0.250000,-0.083333,0.250000,"Chico, CA",17020,2022-11-20 14:36:43


In [6]:
# read US zip code data file
# source: https://github.com/zauberware/postal-codes-json-xml-csv/tree/master/data
# Updated 10 months ago

zip_df = pd.read_csv('data\zipcodes.csv')
texas_zip = zip_df.loc[zip_df['state'] == 'Texas']
print('Num of rows: ', len(texas_zip))
print('Num of columns: ', len(texas_zip.columns))

texas_zip.head()

Num of rows:  2600
Num of columns:  11


,country_code,zipcode,place,state,state_code,province,province_code,community,community_code,latitude,longitude
33846,US,75763,Frankston,Texas,TX,Anderson,1.0,NaN,NaN,32.0535,-95.5163
33847,US,75779,Neches,Texas,TX,Anderson,1.0,NaN,NaN,31.8668,-95.4958
33848,US,75801,Palestine,Texas,TX,Anderson,1.0,NaN,NaN,31.7588,-95.6342
33849,US,75802,Palestine,Texas,TX,Anderson,1.0,NaN,NaN,31.7621,-95.6308
33850,US,75803,Palestine,Texas,TX,Anderson,1.0,NaN,NaN,31.7571,-95.6545


In [ ]:
#Filter data and clean-up

#filter real estate dataset on Texas
texas_df = df.loc[df['state'] == 'Texas']

#drop columns we don't need
texas_df.drop(texas_df.columns[[2, 3, 4, 5, 6, 8, 37, 38, 39]], axis=1, inplace=True)

#region column isolate zipcode into new column 'zipcode'
texas_df['zipcode'] = texas_df.apply(lambda x: x['region'].split(':')[1].strip(), axis=1)
texas_df['zipcode'] = texas_df['zipcode'].astype(int)

#should check that zip codes are valid and that when we merge/join we aren't losing any data
#looks like around 1000 rows are dropped after the merge

#merge zip code data to add zipcode, city, county fields to original Redfin Real estate data. 
texas_merge_df = pd.merge(
    texas_df,
    texas_zip[['zipcode','place','province']],
    how='inner',
    on=['zipcode']
)

#rename columns
rename_cols = {'place': 'city',
        'province': 'county'}

texas_merge_df.rename(columns=rename_cols,
          inplace=True)

#list if counties to focus on
counties_list = ['Williamson', 'Travis', 'Hays', 'Bastrop', 'Caldwell']

#filter texas dataset to counties we are interested in
texas_counties_df = texas_merge_df.loc[texas_merge_df['county'].isin(counties_list)]

property_type = ['Multi-Family (2-4 Unit)']
texas_counties_df = texas_counties_df.loc[~texas_counties_df['property_type'].isin(property_type)]

#csv export
#texas_counties_df.to_csv('redfin_texas_counties.csv')

In [31]:
#preview
print('Num of rows:', len(texas_counties_df))
texas_counties_df.head()

Num of rows: 31549


,period_begin,period_end,region,state,state_code,property_type,property_type_id,median_sale_price,median_sale_price_mom,median_sale_price_yoy,...,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated,zipcode,city,county
3883,2014-07-01,2014-09-30,Zip Code: 78729,Texas,TX,All Residential,-1,243000.0,-0.005932,0.107314,...,NaN,0.400000,0.066667,-0.118519,"Austin, TX",12420,2022-11-20 14:36:43,78729,Austin,Williamson
3884,2019-05-01,2019-07-31,Zip Code: 78729,Texas,TX,Condo/Co-op,3,276000.0,0.232143,0.419023,...,-0.25,0.666667,-0.333333,NaN,"Austin, TX",12420,2022-11-20 14:36:43,78729,Austin,Williamson
3885,2019-07-01,2019-09-30,Zip Code: 78729,Texas,TX,Townhouse,13,312463.0,0.046020,0.227752,...,NaN,NaN,NaN,NaN,"Austin, TX",12420,2022-11-20 14:36:43,78729,Austin,Williamson
3886,2016-10-01,2016-12-31,Zip Code: 78729,Texas,TX,Condo/Co-op,3,212000.0,-0.004695,-0.023041,...,NaN,0.500000,0.100000,NaN,"Austin, TX",12420,2022-11-20 14:36:43,78729,Austin,Williamson
3887,2013-09-01,2013-11-30,Zip Code: 78729,Texas,TX,Condo/Co-op,3,140250.0,0.050562,0.002502,...,-0.50,0.500000,-0.500000,NaN,"Austin, TX",12420,2022-11-20 14:36:43,78729,Austin,Williamson


In [12]:
texas_counties_df.info()
texas_counties_df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36694 entries, 3883 to 352612
Data columns (total 52 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   period_begin                    36694 non-null  object 
 1   period_end                      36694 non-null  object 
 2   region                          36694 non-null  object 
 3   state                           36694 non-null  object 
 4   state_code                      36694 non-null  object 
 5   property_type                   36694 non-null  object 
 6   property_type_id                36694 non-null  int64  
 7   median_sale_price               36694 non-null  float64
 8   median_sale_price_mom           35328 non-null  float64
 9   median_sale_price_yoy           33193 non-null  float64
 10  median_list_price               34670 non-null  float64
 11  median_list_price_mom           33219 non-null  float64
 12  median_list_price_yoy       

,property_type_id,median_sale_price,median_sale_price_mom,median_sale_price_yoy,median_list_price,median_list_price_mom,median_list_price_yoy,median_ppsf,median_ppsf_mom,median_ppsf_yoy,...,sold_above_list_mom,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region_metro_code,zipcode
count,36694.000000,3.669400e+04,35328.000000,33193.000000,3.467000e+04,33219.000000,31536.000000,36690.000000,35324.000000,33188.000000,...,35328.000000,33193.000000,25827.000000,23249.000000,21598.000000,29533.000000,26598.000000,26081.000000,36694.000000,36694.000000
mean,3.789148,3.499373e+05,0.018734,0.155398,3.701842e+05,0.041076,0.168039,193.231848,0.014200,0.140672,...,0.000372,0.033266,0.586626,0.008908,-0.001757,0.433720,-0.002602,-0.001654,12634.648716,78615.785524
std,4.091990,2.209501e+05,0.177104,0.400311,2.453578e+05,2.724775,2.673111,122.643604,0.138051,0.557558,...,0.137281,0.266363,0.236425,0.185772,0.246642,0.284525,0.304983,0.313095,2497.801108,430.555690
min,-1.000000,2.750000e+03,-0.996642,-0.996382,1.300000e+03,-0.997840,-0.998071,1.286249,-0.993643,-0.993149,...,-1.000000,-1.000000,0.020000,-0.875000,-0.952381,0.000000,-1.000000,-1.000000,12420.000000,76527.000000
25%,-1.000000,2.070000e+05,-0.017335,0.017647,2.199000e+05,-0.019737,0.014146,122.561359,-0.009943,0.028887,...,-0.027778,-0.048469,0.417910,-0.079406,-0.136657,0.250000,-0.129533,-0.141324,12420.000000,78648.000000
50%,4.000000,2.950000e+05,0.000000,0.097143,3.100000e+05,0.000000,0.089285,161.959221,0.000957,0.089781,...,0.000000,0.002994,0.578947,0.006352,0.000000,0.435714,0.000000,0.000000,12420.000000,78720.000000
75%,6.000000,4.320000e+05,0.034556,0.219958,4.500000e+05,0.036628,0.202802,237.448547,0.026829,0.188508,...,0.029412,0.113321,0.750000,0.100733,0.133887,0.600000,0.120728,0.141700,12420.000000,78744.000000
max,13.000000,3.875000e+06,5.235731,14.825243,8.000000e+06,454.555556,257.322692,8021.944405,5.630782,58.622673,...,1.000000,1.000000,1.000000,0.875000,0.947368,1.000000,1.000000,1.000000,41700.000000,78957.000000


In [ ]:
#Do some discovery of data here

#might be because we're pulling in quarterly data, try exporting monthly supply from site and importing through CSV?

#Last updated = 11/20/2022 
#can likely pull in data through end of november if we update it